<div class="alert alert-block alert-info">
The model had no improv with CNN + BiLSTM + KAN ever after tweaking the learning rate + decay, batch size, dropout or even the concat which was cat(c3/5/7) + cat(final_layer)
</div>

<div class="alert alert-block alert-info">
Given the augmented dataset N<sub>1</sub> = 400/10K and N<sub>orig</sub> = 40/10K Train/Test Examples, the performance dropped for BEST_SMALL from 89.38% to <85% (84.47%@Val/EP6). 
Model clearly overfits and has no learning capabilities!
</div>

In [ ]:
# import torch
import torch.nn as nn
import torch.nn.functional as F

# Test with BIG + BiLSTM
# ACC - 89.05%
# Not faring well with aug data - tops @ 88.5%

class CustomCNN(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super(CustomCNN, self).__init__()
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        
        # Pre-convolution layer
        self.conv_pre = nn.Conv2d(1, 128, (1, embed_dim), padding=(0, 0))
        self.bn_pre = nn.BatchNorm2d(128)
        self.pool_pre = nn.MaxPool2d((2, 1))
        self.dropout_pre = nn.Dropout(0.1)
        
        self.flatten_pre = nn.Flatten()
        
        # Conv layers with different kernel sizes for 1D input
        self.conv1 = nn.Conv1d(128, 256, 3, padding=1)
        self.bn1 = nn.BatchNorm1d(256)
        self.pool1 = nn.MaxPool1d(2)
        self.dropout1 = nn.Dropout(0.25) 
        
        self.conv2 = nn.Conv1d(128, 256, 5, padding=2)
        self.bn2 = nn.BatchNorm1d(256)
        self.pool2 = nn.MaxPool1d(2)
        self.dropout2 = nn.Dropout(0.25) 
        
        self.conv3 = nn.Conv1d(128, 256, 7, padding=3)
        self.bn3 = nn.BatchNorm1d(256)
        self.pool3 = nn.MaxPool1d(2)
        self.dropout3 = nn.Dropout(0.25) 
        
        # Post PCAT Conv 1
        self.final_conv = nn.Conv2d(3 * 256, 1024, (3, 1), padding=(1, 0))
        
        self.bn_final = nn.BatchNorm2d(1024)
        self.pool_final = nn.MaxPool2d((2, 1))
        self.dropout_pcat1 = nn.Dropout(0.25)

        # # Post PCAT Conv 2
        # self.conv_pcat2 = nn.Conv2d(16384, 32768, (3, 3), padding=(1, 1))

        # self.bn_pcat2 = nn.BatchNorm2d(32768)
        # self.pool_pcat2 = nn.MaxPool2d((2, 1))
        # self.dropout_pcat2 = nn.Dropout(0.15)

        # Post PCAT Conv 3
        self.conv_pcat3 = nn.Conv2d(1024, 2048, (3, 1), padding=(1, 0))

        self.bn_pcat3 = nn.BatchNorm2d(2048)
        self.pool_pcat3 = nn.MaxPool2d((2, 1))
        self.dropout_pcat3 = nn.Dropout(0.25)

        # Post PCAT Conv 4
        self.conv_pcat4 = nn.Conv2d(2048, 4096, (3, 1), padding=(1, 0))
        self.bn_pcat4 = nn.BatchNorm2d(4096)
        self.pool_pcat4 = nn.MaxPool2d((4, 1))
        self.dropout_pcat4 = nn.Dropout(0.25)

        # Post PCAT Conv 5
        self.conv_pcat5 = nn.Conv2d(4096, 8192, (3, 1), padding=(1, 0))

        self.bn_pcat5 = nn.BatchNorm2d(8192)
        self.pool_pcat5 = nn.MaxPool2d((4, 1))
        self.dropout_pcat5 = nn.Dropout(0.6)

        self.flatten_post = nn.Flatten()

        # BiLSTM
        self.bilstm = nn.LSTM(input_size=24576, hidden_size=256, num_layers=1, bidirectional=True, batch_first=True)
        
        # Replace dense layers with FastKAN
        self.kan = KAN([256*2, 256, 64, 2], num_grids=8)
        
    def forward(self, x):
        # x shape: (batch_size, seq_len)
        
        # Embedding
        x = self.embedding(x)  # shape: (batch_size, seq_len, embed_dim)
        x = x.unsqueeze(1)  # shape: (batch_size, 1, seq_len, embed_dim)
        
        # Pre-convolution layer
        x = self.conv_pre(x)  # shape: (batch_size, 1024, seq_len, 1)
        x = self.bn_pre(x)
        x = F.relu(x)
        x = self.pool_pre(x)  # shape: (batch_size, 256, seq_len // 2)
        x = self.dropout_pre(x)
        x = x.squeeze(3)  # shape: (batch_size, 1024, seq_len)
        
        x_flat_pre = self.flatten_pre(x)
        
        # Conv layer 1
        x1 = self.conv1(x)  # shape: (batch_size, 256, seq_len)
        x1 = self.bn1(x1)
        x1 = F.relu(x1)
        x1 = self.pool1(x1)  # shape: (batch_size, 256, seq_len // 2)
        x1 = self.dropout1(x1)
        
        # Conv layer 2
        x2 = self.conv2(x)  # shape: (batch_size, 256, seq_len)
        x2 = self.bn2(x2)
        x2 = F.relu(x2)
        x2 = self.pool2(x2)  # shape: (batch_size, 256, seq_len // 2)
        x2 = self.dropout2(x2)
        
        # Conv layer 3
        x3 = self.conv3(x)  # shape: (batch_size, 256, seq_len)
        x3 = self.bn3(x3)
        x3 = F.relu(x3)
        x3 = self.pool3(x3)  # shape: (batch_size, 256, seq_len // 2)
        x3 = self.dropout3(x3) 
        
        # Concatenate along the channel dimension
        x_cat = torch.cat((x1, x2, x3), dim=1)  # shape: (batch_size, 768, seq_len // 2)
        x_cat = x_cat.unsqueeze(3)  # shape: (batch_size, 768, seq_len // 2, 1)
        
        # PCAT1 Conv2D layer
        x_out = self.final_conv(x_cat)  # shape: (batch_size, 1024, seq_len // 2, 1)
        x_out = self.bn_final(x_out)
        x_out = F.relu(x_out)
        x_out = self.pool_final(x_out)  # shape: (batch_size, 1024, seq_len // 4, 1)

        x_out = self.dropout_pcat1(x_out)

        # # PCAT2 Conv2D layer
        # x_out = self.conv_pcat2(x_out)  # shape: (batch_size, 2048, seq_len // 4, 1)
        # x_out = self.bn_pcat2(x_out)
        # x_out = F.relu(x_out)
        # x_out = self.pool_pcat2(x_out)  # shape: (batch_size, 2048, seq_len // 8, 1)

        # PCAT3 Conv2D layer
        x_out = self.conv_pcat3(x_out)  # shape: (batch_size, 4096, seq_len // 8, 1)
        x_out = self.bn_pcat3(x_out)
        x_out = F.relu(x_out)
        x_out = self.pool_pcat3(x_out)  # shape: (batch_size, 4096, seq_len // 32, 1)
        
        x_out = self.dropout_pcat3(x_out)

        # PCAT4 Conv2D layer
        x_out = self.conv_pcat4(x_out)  # shape: (batch_size, 4096, seq_len // 8, 1)
        x_out = self.bn_pcat4(x_out)
        x_out = F.relu(x_out)
        x_out = self.pool_pcat4(x_out)  # shape: (batch_size, 4096, seq_len // 32, 1)
        
        x_out = self.dropout_pcat4(x_out)

        # PCAT5 Conv2D layer
        x_out = self.conv_pcat5(x_out)  # shape: (batch_size, 4096, seq_len // 8, 1)
        x_out = self.bn_pcat5(x_out)
        x_out = F.relu(x_out)
        x_out = self.pool_pcat5(x_out)  # shape: (batch_size, 4096, seq_len // 32, 1)
        
        x_out = self.dropout_pcat5(x_out)
        
        # Flatten the tensor for the dense layer
        # x_out = x_out.view(x_out.size(0), -1)  # shape: (batch_size, 4096 * (seq_len // 32))
        x_flat_post = self.flatten_post(x_out)

        x_conv_final = torch.cat((x_flat_pre, x_flat_post), dim=1)

        # Pass through BiLSTM
        bilstm_out, _ = self.bilstm(x_conv_final)  # bilstm_out shape: (batch_size, seq_len, hidden_size*2)

        # Use the last hidden state from both directions
        bilstm_out = bilstm_out.squeeze(1)
        
        # Dense layer
        x_out = self.kan(bilstm_out)  # shape: (batch_size, num_classes)
        
        return x_out

# Hyperparams
vocab_size = len(tokenizer)  # Vocabulary size of GPT-2 tokenizer
embed_dim = 64  # Embedding dimension
num_class = 2  # Number of classes (negative, positive)

model = CustomCNN(vocab_size, embed_dim)
model.to(device)

In [ ]:
# import torch
import torch.nn as nn
import torch.nn.functional as F

# ACC 88.8%
# For the next model I found I don't need the conv2d, since after the merge I deal with (N x 1) so the whole kernel = (3, 1) was pointless
# NOTE for myself: Don't work for too long... you make v v stupid mistakes

class CustomCNN(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super(CustomCNN, self).__init__()
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        
        # Pre-convolution layer
        self.conv_pre = nn.Conv2d(1, 128, (1, embed_dim), padding=(0, 0))
        self.bn_pre = nn.BatchNorm2d(128)
        self.pool_pre = nn.MaxPool2d((2, 1))
        self.dropout_pre = nn.Dropout(0.1)
        
        # Conv layers with different kernel sizes for 1D input
        self.conv1 = nn.Conv1d(128, 256, 1, padding=0)
        self.bn1 = nn.BatchNorm1d(256)
        self.pool1 = nn.MaxPool1d(2)
        self.dropout1 = nn.Dropout(0.25) 
        
        self.conv2 = nn.Conv1d(128, 256, 3, padding=1)
        self.bn2 = nn.BatchNorm1d(256)
        self.pool2 = nn.MaxPool1d(2)
        self.dropout2 = nn.Dropout(0.25) 
        
        self.conv3 = nn.Conv1d(128, 256, 5, padding=2)
        self.bn3 = nn.BatchNorm1d(256)
        self.pool3 = nn.MaxPool1d(2)
        self.dropout3 = nn.Dropout(0.25) 
        
        # Post PCAT Conv 1
        self.final_conv = nn.Conv2d(3 * 256, 1536, (3, 3), padding=(1, 1))
        
        self.bn_final = nn.BatchNorm2d(1536)
        self.pool_final = nn.MaxPool2d((2, 1))
        self.dropout_pcat1 = nn.Dropout(0.25)

        # # Post PCAT Conv 2
        # self.conv_pcat2 = nn.Conv2d(16384, 32768, (3, 3), padding=(1, 1))

        # self.bn_pcat2 = nn.BatchNorm2d(32768)
        # self.pool_pcat2 = nn.MaxPool2d((2, 1))
        # self.dropout_pcat2 = nn.Dropout(0.15)

        # Post PCAT Conv 3
        self.conv_pcat3 = nn.Conv2d(1536, 3072, (3, 3), padding=(1, 1))

        self.bn_pcat3 = nn.BatchNorm2d(3072)
        self.pool_pcat3 = nn.MaxPool2d((2, 1))
        self.dropout_pcat3 = nn.Dropout(0.25)

        # Post PCAT Conv 4
        self.conv_pcat4 = nn.Conv2d(3072, 6144, (3, 3), padding=(1, 1))
        self.bn_pcat4 = nn.BatchNorm2d(6144)
        self.pool_pcat4 = nn.MaxPool2d((2, 1))
        self.dropout_pcat4 = nn.Dropout(0.6)

        # Post PCAT Conv 5
        # self.conv_pcat5 = nn.Conv2d(6144, 12288, (3, 3), padding=(1, 1))

        # self.bn_pcat5 = nn.BatchNorm2d(12288)
        # self.pool_pcat5 = nn.MaxPool2d((2, 1))
        # self.dropout_pcat5 = nn.Dropout(0.6)
        
        # Replace dense layers with FastKAN
        self.kan = KAN([49152, 16, 4, 2], num_grids=3)
        
    def forward(self, x):
        # x shape: (batch_size, seq_len)
        
        # Embedding
        x = self.embedding(x)  # shape: (batch_size, seq_len, embed_dim)
        x = x.unsqueeze(1)  # shape: (batch_size, 1, seq_len, embed_dim)
        
        # Pre-convolution layer
        x = self.conv_pre(x)  # shape: (batch_size, 1024, seq_len, 1)
        x = self.bn_pre(x)
        x = F.relu(x)
        x = self.pool_pre(x)  # shape: (batch_size, 256, seq_len // 2)
        x = self.dropout_pre(x)
        x = x.squeeze(3)  # shape: (batch_size, 1024, seq_len)
        
        # Conv layer 1
        x1 = self.conv1(x)  # shape: (batch_size, 256, seq_len)
        x1 = self.bn1(x1)
        x1 = F.relu(x1)
        x1 = self.pool1(x1)  # shape: (batch_size, 256, seq_len // 2)
        x1 = self.dropout1(x1)
        
        # Conv layer 2
        x2 = self.conv2(x)  # shape: (batch_size, 256, seq_len)
        x2 = self.bn2(x2)
        x2 = F.relu(x2)
        x2 = self.pool2(x2)  # shape: (batch_size, 256, seq_len // 2)
        x2 = self.dropout2(x2)
        
        # Conv layer 3
        x3 = self.conv3(x)  # shape: (batch_size, 256, seq_len)
        x3 = self.bn3(x3)
        x3 = F.relu(x3)
        x3 = self.pool3(x3)  # shape: (batch_size, 256, seq_len // 2)
        x3 = self.dropout3(x3) 
        
        # Concatenate along the channel dimension
        x_cat = torch.cat((x1, x2, x3), dim=1)  # shape: (batch_size, 768, seq_len // 2)
        x_cat = x_cat.unsqueeze(3)  # shape: (batch_size, 768, seq_len // 2, 1)
        
        # PCAT1 Conv2D layer
        x_out = self.final_conv(x_cat)  # shape: (batch_size, 1024, seq_len // 2, 1)
        x_out = self.bn_final(x_out)
        x_out = F.relu(x_out)
        x_out = self.pool_final(x_out)  # shape: (batch_size, 1024, seq_len // 4, 1)

        x_out = self.dropout_pcat1(x_out)

        # # PCAT2 Conv2D layer
        # x_out = self.conv_pcat2(x_out)  # shape: (batch_size, 2048, seq_len // 4, 1)
        # x_out = self.bn_pcat2(x_out)
        # x_out = F.relu(x_out)
        # x_out = self.pool_pcat2(x_out)  # shape: (batch_size, 2048, seq_len // 8, 1)

        # PCAT3 Conv2D layer
        x_out = self.conv_pcat3(x_out)  # shape: (batch_size, 4096, seq_len // 8, 1)
        x_out = self.bn_pcat3(x_out)
        x_out = F.relu(x_out)
        x_out = self.pool_pcat3(x_out)  # shape: (batch_size, 4096, seq_len // 32, 1)
        
        x_out = self.dropout_pcat3(x_out)

        # PCAT4 Conv2D layer
        x_out = self.conv_pcat4(x_out)  # shape: (batch_size, 4096, seq_len // 8, 1)
        x_out = self.bn_pcat4(x_out)
        x_out = F.relu(x_out)
        x_out = self.pool_pcat4(x_out)  # shape: (batch_size, 4096, seq_len // 32, 1)
        
        x_out = self.dropout_pcat4(x_out)
        
        # Flatten the tensor for the dense layer
        x_out = x_out.view(x_out.size(0), -1)  # shape: (batch_size, 4096 * (seq_len // 32))
        
        # Dense layer
        x_out = self.kan(x_out)  # shape: (batch_size, num_classes)
        
        return x_out

# Hyperparams
vocab_size = len(tokenizer)  # Vocabulary size of GPT-2 tokenizer
embed_dim = 64  # Embedding dimension
num_class = 2  # Number of classes (negative, positive)

model = CustomCNN(vocab_size, embed_dim)
model.to(device)


In [ ]:
# CHANGES - Batch size 32 -> 64
# RESULT, almost nothing 0.8876 ACC

# Batch the data
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Epoch 20, Loss: 0.0011186991391703485, Accuracy: 0.981075
# Val_Loss: 0.005742809381335974, Val_Accuracy: 0.881

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Increases Conv Dims
# Decreases KANs
# Overall performance comparable to previous model
# ACC 88.81%

class CustomCNN(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super(CustomCNN, self).__init__()
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        
        # Pre-convolution layer
        self.conv_pre = nn.Conv2d(1, 256, (1, embed_dim), padding=(0, 0))
        
        # Conv layers with different kernel sizes for 1D input
        self.conv1 = nn.Conv1d(256, 512, 1, padding=0)
        self.bn1 = nn.BatchNorm1d(512)
        self.pool1 = nn.MaxPool1d(2)
        self.dropout1 = nn.Dropout(0.25) 
        
        self.conv2 = nn.Conv1d(256, 512, 3, padding=1)
        self.bn2 = nn.BatchNorm1d(512)
        self.pool2 = nn.MaxPool1d(2)
        self.dropout2 = nn.Dropout(0.25) 
        
        self.conv3 = nn.Conv1d(256, 512, 5, padding=2)
        self.bn3 = nn.BatchNorm1d(512)
        self.pool3 = nn.MaxPool1d(2)
        self.dropout3 = nn.Dropout(0.25) 
        
        # Post PCAT Conv 1
        self.final_conv = nn.Conv2d(3 * 512, 4096, (3, 3), padding=(1, 1))
        
        self.bn_final = nn.BatchNorm2d(4096)
        self.pool_final = nn.MaxPool2d((2, 1))
        self.dropout_pcat1 = nn.Dropout(0.25)

        # # Post PCAT Conv 2
        # self.conv_pcat2 = nn.Conv2d(16384, 32768, (3, 3), padding=(1, 1))

        # self.bn_pcat2 = nn.BatchNorm2d(32768)
        # self.pool_pcat2 = nn.MaxPool2d((2, 1))
        # self.dropout_pcat2 = nn.Dropout(0.15)

        # Post PCAT Conv 3
        self.conv_pcat3 = nn.Conv2d(4096, 8192, (3, 3), padding=(1, 1))

        self.bn_pcat3 = nn.BatchNorm2d(8192)
        self.pool_pcat3 = nn.MaxPool2d((2, 1))
        self.dropout_pcat3 = nn.Dropout(0.5)
        
        # Replace dense layers with FastKAN
        self.kan = KAN([262144, 50, 6, 2], num_grids=3)
        
    def forward(self, x):
        # x shape: (batch_size, seq_len)
        
        # Embedding
        x = self.embedding(x)  # shape: (batch_size, seq_len, embed_dim)
        x = x.unsqueeze(1)  # shape: (batch_size, 1, seq_len, embed_dim)
        
        # Pre-convolution layer
        x = self.conv_pre(x)  # shape: (batch_size, 1024, seq_len, 1)
        x = x.squeeze(3)  # shape: (batch_size, 1024, seq_len)
        
        # Conv layer 1
        x1 = self.conv1(x)  # shape: (batch_size, 256, seq_len)
        x1 = self.bn1(x1)
        x1 = F.relu(x1)
        x1 = self.pool1(x1)  # shape: (batch_size, 256, seq_len // 2)
        x1 = self.dropout1(x1)
        
        # Conv layer 2
        x2 = self.conv2(x)  # shape: (batch_size, 256, seq_len)
        x2 = self.bn2(x2)
        x2 = F.relu(x2)
        x2 = self.pool2(x2)  # shape: (batch_size, 256, seq_len // 2)
        x2 = self.dropout2(x2)
        
        # Conv layer 3
        x3 = self.conv3(x)  # shape: (batch_size, 256, seq_len)
        x3 = self.bn3(x3)
        x3 = F.relu(x3)
        x3 = self.pool3(x3)  # shape: (batch_size, 256, seq_len // 2)
        x3 = self.dropout3(x3) 
        
        # Concatenate along the channel dimension
        x_cat = torch.cat((x1, x2, x3), dim=1)  # shape: (batch_size, 768, seq_len // 2)
        x_cat = x_cat.unsqueeze(3)  # shape: (batch_size, 768, seq_len // 2, 1)
        
        # PCAT1 Conv2D layer
        x_out = self.final_conv(x_cat)  # shape: (batch_size, 1024, seq_len // 2, 1)
        x_out = self.bn_final(x_out)
        x_out = F.relu(x_out)
        x_out = self.pool_final(x_out)  # shape: (batch_size, 1024, seq_len // 4, 1)

        x_out = self.dropout_pcat1(x_out)

        # # PCAT2 Conv2D layer
        # x_out = self.conv_pcat2(x_out)  # shape: (batch_size, 2048, seq_len // 4, 1)
        # x_out = self.bn_pcat2(x_out)
        # x_out = F.relu(x_out)
        # x_out = self.pool_pcat2(x_out)  # shape: (batch_size, 2048, seq_len // 8, 1)

        # PCAT3 Conv2D layer
        x_out = self.conv_pcat3(x_out)  # shape: (batch_size, 4096, seq_len // 8, 1)
        x_out = self.bn_pcat3(x_out)
        x_out = F.relu(x_out)
        x_out = self.pool_pcat3(x_out)  # shape: (batch_size, 4096, seq_len // 32, 1)
        
        # Flatten the tensor for the dense layer
        x_out = x_out.view(x_out.size(0), -1)  # shape: (batch_size, 4096 * (seq_len // 32))
        x_out = self.dropout_pcat3(x_out)
        
        # Dense layer
        x_out = self.kan(x_out)  # shape: (batch_size, num_classes)
        
        return x_out

# Hyperparams
vocab_size = len(tokenizer)  # Vocabulary size of GPT-2 tokenizer
embed_dim = 64  # Embedding dimension
num_class = 2  # Number of classes (negative, positive)

model = CustomCNN(vocab_size, embed_dim)
model.to(device)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Model tested for pre-conv over 1 word then passed to Conv 1-3-5
# Model perf is stable (not overfitting) but would be desirable to capture more
# ACC - 88.92% @ Epoch 6

class CustomCNN(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super(CustomCNN, self).__init__()
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        
        # Pre-convolution layer
        self.conv_pre = nn.Conv2d(1, 256, (1, embed_dim), padding=(0, 0))
        
        # Conv layers with different kernel sizes for 1D input
        self.conv1 = nn.Conv1d(256, 512, 1, padding=0)
        self.bn1 = nn.BatchNorm1d(512)
        self.pool1 = nn.MaxPool1d(2)
        self.dropout1 = nn.Dropout(0.25) 
        
        self.conv2 = nn.Conv1d(256, 512, 3, padding=1)
        self.bn2 = nn.BatchNorm1d(512)
        self.pool2 = nn.MaxPool1d(2)
        self.dropout2 = nn.Dropout(0.25) 
        
        self.conv3 = nn.Conv1d(256, 512, 5, padding=2)
        self.bn3 = nn.BatchNorm1d(512)
        self.pool3 = nn.MaxPool1d(2)
        self.dropout3 = nn.Dropout(0.25) 
        
        # Post PCAT Conv 1
        self.final_conv = nn.Conv2d(3 * 512, 4096, (3, 3), padding=(1, 1))
        
        self.bn_final = nn.BatchNorm2d(4096)
        self.pool_final = nn.MaxPool2d((2, 1))
        self.dropout_pcat1 = nn.Dropout(0.15)

        # # Post PCAT Conv 2
        # self.conv_pcat2 = nn.Conv2d(16384, 32768, (3, 3), padding=(1, 1))

        # self.bn_pcat2 = nn.BatchNorm2d(32768)
        # self.pool_pcat2 = nn.MaxPool2d((2, 1))
        # self.dropout_pcat2 = nn.Dropout(0.15)

        # Post PCAT Conv 3
        self.conv_pcat3 = nn.Conv2d(4096, 8192, (3, 3), padding=(1, 1))

        self.bn_pcat3 = nn.BatchNorm2d(8192)
        self.pool_pcat3 = nn.MaxPool2d((4, 1))
        self.dropout_pcat3 = nn.Dropout(0.15)
        
        # Replace dense layers with FastKAN
        self.kan = KAN([131072, 64, 8, 2], num_grids=3)
        
    def forward(self, x):
        # x shape: (batch_size, seq_len)
        
        # Embedding
        x = self.embedding(x)  # shape: (batch_size, seq_len, embed_dim)
        x = x.unsqueeze(1)  # shape: (batch_size, 1, seq_len, embed_dim)
        
        # Pre-convolution layer
        x = self.conv_pre(x)  # shape: (batch_size, 1024, seq_len, 1)
        x = x.squeeze(3)  # shape: (batch_size, 1024, seq_len)
        
        # Conv layer 1
        x1 = self.conv1(x)  # shape: (batch_size, 256, seq_len)
        x1 = self.bn1(x1)
        x1 = F.relu(x1)
        x1 = self.pool1(x1)  # shape: (batch_size, 256, seq_len // 2)
        x1 = self.dropout1(x1)
        
        # Conv layer 2
        x2 = self.conv2(x)  # shape: (batch_size, 256, seq_len)
        x2 = self.bn2(x2)
        x2 = F.relu(x2)
        x2 = self.pool2(x2)  # shape: (batch_size, 256, seq_len // 2)
        x2 = self.dropout2(x2)
        
        # Conv layer 3
        x3 = self.conv3(x)  # shape: (batch_size, 256, seq_len)
        x3 = self.bn3(x3)
        x3 = F.relu(x3)
        x3 = self.pool3(x3)  # shape: (batch_size, 256, seq_len // 2)
        x3 = self.dropout3(x3) 
        
        # Concatenate along the channel dimension
        x_cat = torch.cat((x1, x2, x3), dim=1)  # shape: (batch_size, 768, seq_len // 2)
        x_cat = x_cat.unsqueeze(3)  # shape: (batch_size, 768, seq_len // 2, 1)
        
        # PCAT1 Conv2D layer
        x_out = self.final_conv(x_cat)  # shape: (batch_size, 1024, seq_len // 2, 1)
        x_out = self.bn_final(x_out)
        x_out = F.relu(x_out)
        x_out = self.pool_final(x_out)  # shape: (batch_size, 1024, seq_len // 4, 1)

        x_out = self.dropout_pcat1(x_out)

        # # PCAT2 Conv2D layer
        # x_out = self.conv_pcat2(x_out)  # shape: (batch_size, 2048, seq_len // 4, 1)
        # x_out = self.bn_pcat2(x_out)
        # x_out = F.relu(x_out)
        # x_out = self.pool_pcat2(x_out)  # shape: (batch_size, 2048, seq_len // 8, 1)

        # PCAT3 Conv2D layer
        x_out = self.conv_pcat3(x_out)  # shape: (batch_size, 4096, seq_len // 8, 1)
        x_out = self.bn_pcat3(x_out)
        x_out = F.relu(x_out)
        x_out = self.pool_pcat3(x_out)  # shape: (batch_size, 4096, seq_len // 32, 1)
        
        # Flatten the tensor for the dense layer
        x_out = x_out.view(x_out.size(0), -1)  # shape: (batch_size, 4096 * (seq_len // 32))
        x_out = self.dropout_pcat3(x_out)
        
        # Dense layer
        x_out = self.kan(x_out)  # shape: (batch_size, num_classes)
        
        return x_out

# Hyperparams
vocab_size = len(tokenizer)  # Vocabulary size of GPT-2 tokenizer
embed_dim = 32  # Embedding dimension
num_class = 2  # Number of classes (negative, positive)

model = CustomCNN(vocab_size, embed_dim)
model.to(device)


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# BEST__SMALL
# Model 89.38% ACC
class CustomCNN(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super(CustomCNN, self).__init__()
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        
        # Conv layers with different kernel sizes
        self.conv1 = nn.Conv2d(1, 256, (1, embed_dim), padding=(0, 0))
        self.bn1 = nn.BatchNorm2d(256)
        self.pool1 = nn.MaxPool1d(2)
        self.dropout1 = nn.Dropout(0.25) 
        
        self.conv2 = nn.Conv2d(1, 256, (3, embed_dim), padding=(1, 0))
        self.bn2 = nn.BatchNorm2d(256)
        self.pool2 = nn.MaxPool1d(2)
        self.dropout2 = nn.Dropout(0.25) 
        
        self.conv3 = nn.Conv2d(1, 256, (5, embed_dim), padding=(2, 0))
        self.bn3 = nn.BatchNorm2d(256)
        self.pool3 = nn.MaxPool1d(2)
        self.dropout3 = nn.Dropout(0.25) 
        
        # Post PCAT Conv 1
        self.final_conv = nn.Conv2d(3 * 256, 1024, (3, 3), padding=(1, 1))
        
        self.bn_final = nn.BatchNorm2d(1024)
        self.pool_final = nn.MaxPool2d((2, 1))
        self.dropout_pcat1 = nn.Dropout(0.25)

        # Post PCAT Conv 2
        self.conv_pcat2 = nn.Conv2d(1024, 2048, (3, 3), padding=(1, 1))

        self.bn_pcat2 = nn.BatchNorm2d(2048)
        self.pool_pcat2 = nn.MaxPool2d((2, 1))
        self.dropout_pcat2 = nn.Dropout(0.25)

        # Post PCAT Conv 3
        self.conv_pcat3 = nn.Conv2d(2048, 4096, (3, 3), padding=(1, 1))

        self.bn_pcat3 = nn.BatchNorm2d(4096)
        self.pool_pcat3 = nn.MaxPool2d((4, 1))
        self.dropout_pcat3 = nn.Dropout(0.25)
        
        # Replace dense layers with FastKAN
        self.kan = KAN([32768, 256, 64, 8, 2], num_grids=8)
        
    def forward(self, x):
        # x shape: (batch_size, seq_len)
        
        # Embedding
        x = self.embedding(x)  # shape: (batch_size, seq_len, embed_dim)
        x = x.unsqueeze(1)  # shape: (batch_size, 1, seq_len, embed_dim)
        
        # Conv layer 1
        x1 = self.conv1(x)  # shape: (batch_size, 100, seq_len-0, 1)
        x1 = self.bn1(x1)
        x1 = F.relu(x1)
        x1 = x1.squeeze(3)  # shape: (batch_size, 100, seq_len-0)
        x1 = self.pool1(x1)  # shape: (batch_size, 100, (seq_len-0)//2)
        x1 = self.dropout1(x1)
        
        # Conv layer 2
        x2 = self.conv2(x)  # shape: (batch_size, 100, seq_len-2, 1)
        x2 = self.bn2(x2)
        x2 = F.relu(x2)
        x2 = x2.squeeze(3)  # shape: (batch_size, 100, seq_len-2)
        x2 = self.pool2(x2)  # shape: (batch_size, 100, (seq_len-2)//2)
        x2 = self.dropout2(x2)
        
        # Conv layer 3
        x3 = self.conv3(x)  # shape: (batch_size, 100, seq_len-4, 1)
        x3 = self.bn3(x3)
        x3 = F.relu(x3)
        x3 = x3.squeeze(3)  # shape: (batch_size, 100, seq_len-4)
        x3 = self.pool3(x3)  # shape: (batch_size, 100, (seq_len-4)//2)
        x3 = self.dropout3(x3) 
        
        # Concatenate along the channel dimension
        x_cat = torch.cat((x1, x2, x3), dim=1)  # shape: (batch_size, 300, ...)
        x_cat = x_cat.unsqueeze(3)  # shape: (batch_size, 300, ..., 1)
        
        # PCAT1 Conv2D layer
        x_out = self.final_conv(x_cat)  # shape: (batch_size, 100, ..., 1)
        x_out = self.bn_final(x_out)
        x_out = F.relu(x_out)
        x_out = self.pool_final(x_out)  # shape: (batch_size, 100, ..., 1)

        x_out = self.dropout_pcat1(x_out)

        # PCAT2 Conv2D layer
        x_out = self.conv_pcat2(x_out)  # shape: (batch_size, 100, ..., 1)
        x_out = self.bn_pcat2(x_out)
        x_out = F.relu(x_out)
        x_out = self.pool_pcat2(x_out)  # shape: (batch_size, 100, ..., 1)

        # PCAT3 Conv2D layer
        x_out = self.conv_pcat3(x_out)  # shape: (batch_size, 100, ..., 1)
        x_out = self.bn_pcat3(x_out)
        x_out = F.relu(x_out)
        x_out = self.pool_pcat3(x_out)  # shape: (batch_size, 100, ..., 1)
        
        # Flatten the tensor for the dense layer
        x_out = x_out.view(x_out.size(0), -1)  # shape: (batch_size, 100)
        x_out = self.dropout_pcat3(x_out)
        
        # Dense layer
        x_out = self.kan(x_out)  # shape: (batch_size, num_classes)
        
        return x_out

# Hyperparams
vocab_size = len(tokenizer)  # Vocabulary size of GPT-2 tokenizer
embed_dim = 32  # Embedding dimension
num_class = 2  # Number of classes (negative, positive)

model = CustomCNN(vocab_size, embed_dim)
model.to(device)

NameError: name 'tokenizer' is not defined

In [ ]:
# 89.0% Val
import torch
import torch.nn as nn
import torch.nn.functional as F

class CustomCNN(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super(CustomCNN, self).__init__()
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        
        # Conv layers with different kernel sizes
        self.conv1 = nn.Conv2d(1, 100, (1, embed_dim), padding=(0, 0))
        self.bn1 = nn.BatchNorm2d(100)
        self.pool1 = nn.MaxPool1d(2)
        
        self.conv2 = nn.Conv2d(1, 100, (3, embed_dim), padding=(1, 0))
        self.bn2 = nn.BatchNorm2d(100)
        self.pool2 = nn.MaxPool1d(2)
        
        self.conv3 = nn.Conv2d(1, 100, (5, embed_dim), padding=(2, 0))
        self.bn3 = nn.BatchNorm2d(100)
        self.pool3 = nn.MaxPool1d(2)
        
        # Final Conv2D layer after concatenation
        self.final_conv = nn.Conv2d(3 * 100, 500, (3, 3), padding=(1, 1))
        
        # Additional layers
        self.bn_final = nn.BatchNorm2d(100)
        self.pool_final = nn.MaxPool2d((2, 1))
        self.dropout = nn.Dropout(0.65)
        
        # Dense layer
        #self.fc = nn.Linear(100, num_classes)
        
        # Replace dense layers with FastKAN
        self.kan = KAN([32000, 512, 128, 2], num_grids=4)
        
    def forward(self, x):
        # x shape: (batch_size, seq_len)
        
        # Embedding
        x = self.embedding(x)  # shape: (batch_size, seq_len, embed_dim)
        x = x.unsqueeze(1)  # shape: (batch_size, 1, seq_len, embed_dim)
        
        # Conv layer 1
        x1 = self.conv1(x)  # shape: (batch_size, 100, seq_len-0, 1)
        x1 = self.bn1(x1)
        x1 = F.relu(x1)
        x1 = x1.squeeze(3)  # shape: (batch_size, 100, seq_len-0)
        x1 = self.pool1(x1)  # shape: (batch_size, 100, (seq_len-0)//2)
        
        # Conv layer 2
        x2 = self.conv2(x)  # shape: (batch_size, 100, seq_len-2, 1)
        x2 = self.bn2(x2)
        x2 = F.relu(x2)
        x2 = x2.squeeze(3)  # shape: (batch_size, 100, seq_len-2)
        x2 = self.pool2(x2)  # shape: (batch_size, 100, (seq_len-2)//2)
        
        # Conv layer 3
        x3 = self.conv3(x)  # shape: (batch_size, 100, seq_len-4, 1)
        x3 = self.bn3(x3)
        x3 = F.relu(x3)
        x3 = x3.squeeze(3)  # shape: (batch_size, 100, seq_len-4)
        x3 = self.pool3(x3)  # shape: (batch_size, 100, (seq_len-4)//2)
        
        # Concatenate along the channel dimension
        x_cat = torch.cat((x1, x2, x3), dim=1)  # shape: (batch_size, 300, ...)
        x_cat = x_cat.unsqueeze(3)  # shape: (batch_size, 300, ..., 1)
        
        # Final Conv2D layer
        x_out = self.final_conv(x_cat)  # shape: (batch_size, 100, ..., 1)
        #x_out = self.bn_final(x_out)
        x_out = F.relu(x_out)
        x_out = self.pool_final(x_out)  # shape: (batch_size, 100, ..., 1)
        
        # Flatten the tensor for the dense layer
        x_out = x_out.view(x_out.size(0), -1)  # shape: (batch_size, 100)
        x_out = self.dropout(x_out)
        
        # Dense layer
        x_out = self.kan(x_out)  # shape: (batch_size, num_classes)
        
        return x_out

# Hyperparams
vocab_size = len(tokenizer)  # Vocabulary size of GPT-2 tokenizer
embed_dim = 128  # Embedding dimension
num_class = 2  # Number of classes (negative, positive)

model = CustomCNN(vocab_size, embed_dim)
model.to(device)

In [ ]:
# OLD
class SentimentCNNModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super(SentimentCNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        
        # convolutional layers with batch Normalization
        self.conv1 = nn.Conv2d(1, 100, (1, embed_dim), padding=(0, 0))  # 1-gram
        self.bn1 = nn.BatchNorm2d(100)
        
        self.conv2 = nn.Conv2d(1, 100, (3, embed_dim), padding=(1, 0))  # 3-gram
        self.bn2 = nn.BatchNorm2d(100)
        
        self.conv3 = nn.Conv2d(1, 100, (5, embed_dim), padding=(2, 0))  # 5-gram
        self.bn3 = nn.BatchNorm2d(100)
        
        self.dropout = nn.Dropout(0.5)

        self.conv_pcat1 = nn.Conv2d(1, 200, (3, 3), padding=(0,0))
        self.bn_pcat1 = nn.BatchNorm2d(200)
        
        
        # Replace dense layers with FastKAN
        self.kan = KAN([37500, 512, 128, num_class], num_grids=4)

    def forward(self, text):
        embedded = self.embedding(text).unsqueeze(1)  # Add channel dimension
        
        conv1_out = F.relu(self.bn1(self.conv1(embedded))).squeeze(3)
        conv2_out = F.relu(self.bn2(self.conv2(embedded))).squeeze(3)
        conv3_out = F.relu(self.bn3(self.conv3(embedded))).squeeze(3)
        
        pooled1 = F.max_pool1d(conv1_out, 2).squeeze(2)
        pooled2 = F.max_pool1d(conv2_out, 2).squeeze(2)
        pooled3 = F.max_pool1d(conv3_out, 2).squeeze(2)

        cat = torch.cat((pooled1, pooled2, pooled3), 0)

        conv_pcat1_out = F.relu(self.bn_pcat1(self.conv_pcat1(cat))).squeeze(3)
        pooled_pcat1 = F.max_pool2d(conv1_out, 2).squeeze(2)
        
        #cat = cat.view(cat.size(0), -1)
        dropout = self.dropout(pooled_pcat1)
        
        # Use kan for final classification
        out = self.kan(dropout)
        return out


# Hyperparams
vocab_size = len(tokenizer)  # Vocabulary size of GPT-2 tokenizer
embed_dim = 128  # Embedding dimension
num_class = 2  # Number of classes (negative, positive)

model = SentimentCNNModel(vocab_size, embed_dim, num_class)
model.to(device)